In [74]:
import numpy as np
import pandas as pd

data = pd.read_csv("./data/2_train.csv", sep=';', index_col='id')
test = pd.read_csv("./data/2_test.csv", sep=';', index_col = 'id')

data_all = pd.concat([data, test], axis=0)
# 将非法值替换成空值
for x in ['-', '.', '?', 'nonexistent']:
    data_all.replace(x, np.nan, inplace=True)
data_all.head(5).style

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
id,,,,,,,,,,,,,,,,,,,,,
1,nan,housemaid,married,basic.4y,nan,no,nan,telephone,may,nan,261,nan,999,0,nan,1.100000,93.994000,-36.400000,4.857000,5191.000000,no
2,57,services,married,high.school,nan,no,no,telephone,may,nan,149,1,999,0,nan,1.100000,93.994000,-36.400000,4.857000,5191.000000,no
3,37,services,nan,high.school,no,nan,no,telephone,may,nan,nan,1,999,0,nan,1.100000,93.994000,-36.400000,4.857000,5191.000000,no
4,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nan,1.100000,93.994000,-36.400000,4.857000,5191.000000,no
5,56,services,married,high.school,no,no,yes,nan,may,mon,307,1,999,nan,nan,1.100000,93.994000,-36.400000,4.857000,5191.000000,no


In [75]:
data_all.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [76]:
def column_mapping_str(X, col_name='', prefix_name='', data_name='X', begin_idx=0):
    """
    根据指定列的值，生成mapping字符串
    """
    col_value_counts = X[col_name].value_counts()
    print("------------%s每个项的分布--------------" % col_name)
    print(col_value_counts)
    print("--------------------------------------")
    label_str = ""
    for idx, key in enumerate(col_value_counts.keys()):
        label_str = "".join([label_str, "'", str(key), "'", ": ", str(idx + begin_idx), ", "])
    label_str = label_str[0:-2]
    print("mapping_%s = {%s}" % (prefix_name, label_str))
    print("%s['%s_out'] = %s['%s'].map(mapping_%s).fillna(-99).astype(int)" % (data_name,  col_name, data_name, col_name, prefix_name))
    print("%s_%s = pd.get_dummies(%s['%s_out'], prefix='%s', drop_first=True)" % (data_name,  prefix_name, data_name,col_name, prefix_name))
    print("%s = pd.concat([%s, pd.DataFrame(%s_%s, index=%s.index)], axis=1)" % (data_name,  data_name, data_name,prefix_name, data_name))
    print("%s = %s.drop(labels='%s', axis=1)" % (data_name, data_name, col_name))
    print("--------------------------------------")

In [77]:
data_all['info_rate'] = (4 - data_all[['age', 'job', 'marital', 'education']].isnull().sum(axis=1)) * 1.0 / 4
data_all['info_rate']

id
1        0.75
2        1.00
3        0.75
4        1.00
5        1.00
         ... 
54114    0.75
54115    0.50
54116    1.00
54117    0.25
54118    0.75
Name: info_rate, Length: 45307, dtype: float64

In [78]:
column_mapping_str(data_all, col_name='job', prefix_name='job', data_name='data_all')
mapping_job = {'admin.': 3, 'blue-collar': 1, 'technician': 1, 'services': 1, 'management': 3,
               'retired': 0, 'self-employed': 1, 'entrepreneur': 3, 'housemaid':0, 'unemployed': 0, 'student': 0}
data_all['job_out'] = data_all['job'].map(mapping_job).fillna(0).astype(int)
data_all_job = pd.get_dummies(data_all['job_out'], prefix='job', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_job, index=data_all.index)], axis=1)
data_all.drop(labels=['job_out'], inplace=True, axis=1)
data_all

------------job每个项的分布--------------
admin.           8570
blue-collar      7607
technician       5521
services         3315
management       2446
retired          1432
self-employed    1198
entrepreneur     1193
housemaid         849
unemployed        848
student           718
Name: job, dtype: int64
--------------------------------------
mapping_job = {'admin.': 0, 'blue-collar': 1, 'technician': 2, 'services': 3, 'management': 4, 'retired': 5, 'self-employed': 6, 'entrepreneur': 7, 'housemaid': 8, 'unemployed': 9, 'student': 10}
data_all['job_out'] = data_all['job'].map(mapping_job).fillna(-99).astype(int)
data_all_job = pd.get_dummies(data_all['job_out'], prefix='job', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_job, index=data_all.index)], axis=1)
data_all = data_all.drop(labels='job', axis=1)
--------------------------------------


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,info_rate,job_1,job_3
id,,,,,,,,,,,,,,,,,,,,,
1,NaN,housemaid,married,basic.4y,NaN,no,NaN,telephone,may,NaN,...,NaN,1.1,93.994,-36.4,4.857,5191.0,no,0.75,0,0
2,57,services,married,high.school,NaN,no,no,telephone,may,NaN,...,NaN,1.1,93.994,-36.4,4.857,5191.0,no,1.00,1,0
3,37,services,NaN,high.school,no,NaN,no,telephone,may,NaN,...,NaN,1.1,93.994,-36.4,4.857,5191.0,no,0.75,1,0
4,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,NaN,1.1,93.994,-36.4,4.857,5191.0,no,1.00,0,1
5,56,services,married,high.school,no,no,yes,NaN,may,mon,...,NaN,1.1,93.994,-36.4,4.857,5191.0,no,1.00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54114,30,admin.,NaN,basic.6y,no,yes,yes,cellular,jul,thu,...,NaN,1.4,93.918,-42.7,4.958,5228.1,Withheld,0.75,0,1
54115,39,admin.,NaN,NaN,no,yes,no,NaN,jul,NaN,...,NaN,1.4,93.918,-42.7,4.959,5228.1,Withheld,0.50,0,1
54116,27,student,single,high.school,no,NaN,no,NaN,NaN,mon,...,failure,-1.8,92.893,-46.2,1.354,5099.1,Withheld,1.00,0,0


In [79]:
column_mapping_str(data_all, col_name='marital', prefix_name='marital', data_name='data_all')
mapping_marital = {'married': 1, 'single': 0, 'divorced': 0}
data_all['marital_out'] = data_all['marital'].map(mapping_marital).fillna(0).astype(int)
data_all

------------marital每个项的分布--------------
married     20704
single       9625
divorced     3783
Name: marital, dtype: int64
--------------------------------------
mapping_marital = {'married': 0, 'single': 1, 'divorced': 2}
data_all['marital_out'] = data_all['marital'].map(mapping_marital).fillna(-99).astype(int)
data_all_marital = pd.get_dummies(data_all['marital_out'], prefix='marital', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_marital, index=data_all.index)], axis=1)
data_all = data_all.drop(labels='marital', axis=1)
--------------------------------------


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,info_rate,job_1,job_3,marital_out
id,,,,,,,,,,,,,,,,,,,,,
1,NaN,housemaid,married,basic.4y,NaN,no,NaN,telephone,may,NaN,...,1.1,93.994,-36.4,4.857,5191.0,no,0.75,0,0,1
2,57,services,married,high.school,NaN,no,no,telephone,may,NaN,...,1.1,93.994,-36.4,4.857,5191.0,no,1.00,1,0,1
3,37,services,NaN,high.school,no,NaN,no,telephone,may,NaN,...,1.1,93.994,-36.4,4.857,5191.0,no,0.75,1,0,0
4,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,1.00,0,1,1
5,56,services,married,high.school,no,no,yes,NaN,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,1.00,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54114,30,admin.,NaN,basic.6y,no,yes,yes,cellular,jul,thu,...,1.4,93.918,-42.7,4.958,5228.1,Withheld,0.75,0,1,0
54115,39,admin.,NaN,NaN,no,yes,no,NaN,jul,NaN,...,1.4,93.918,-42.7,4.959,5228.1,Withheld,0.50,0,1,0
54116,27,student,single,high.school,no,NaN,no,NaN,NaN,mon,...,-1.8,92.893,-46.2,1.354,5099.1,Withheld,1.00,0,0,0


In [80]:
column_mapping_str(data_all, col_name='education', prefix_name='education', data_name='data_all')
mapping_education = {'university.degree': 2, 'high.school': 1, 'basic.9y': 0, 'professional.course': 3,
                     'basic.4y': 0, 'basic.6y': 0, 'illiterate': 0}
data_all['education_out'] = data_all['education'].map(mapping_education).fillna(1).astype(int)
data_all_education = pd.get_dummies(data_all['education_out'], prefix='education', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_education, index=data_all.index)], axis=1)
data_all.drop(labels=['education_out'], inplace=True, axis=1)
data_all

------------education每个项的分布--------------
university.degree      10034
high.school             7884
basic.9y                4964
professional.course     4344
basic.4y                3411
basic.6y                1882
illiterate                16
Name: education, dtype: int64
--------------------------------------
mapping_education = {'university.degree': 0, 'high.school': 1, 'basic.9y': 2, 'professional.course': 3, 'basic.4y': 4, 'basic.6y': 5, 'illiterate': 6}
data_all['education_out'] = data_all['education'].map(mapping_education).fillna(-99).astype(int)
data_all_education = pd.get_dummies(data_all['education_out'], prefix='education', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_education, index=data_all.index)], axis=1)
data_all = data_all.drop(labels='education', axis=1)
--------------------------------------


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,euribor3m,nr.employed,y,info_rate,job_1,job_3,marital_out,education_1,education_2,education_3
id,,,,,,,,,,,,,,,,,,,,,
1,NaN,housemaid,married,basic.4y,NaN,no,NaN,telephone,may,NaN,...,4.857,5191.0,no,0.75,0,0,1,0,0,0
2,57,services,married,high.school,NaN,no,no,telephone,may,NaN,...,4.857,5191.0,no,1.00,1,0,1,1,0,0
3,37,services,NaN,high.school,no,NaN,no,telephone,may,NaN,...,4.857,5191.0,no,0.75,1,0,0,1,0,0
4,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,4.857,5191.0,no,1.00,0,1,1,0,0,0
5,56,services,married,high.school,no,no,yes,NaN,may,mon,...,4.857,5191.0,no,1.00,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54114,30,admin.,NaN,basic.6y,no,yes,yes,cellular,jul,thu,...,4.958,5228.1,Withheld,0.75,0,1,0,0,0,0
54115,39,admin.,NaN,NaN,no,yes,no,NaN,jul,NaN,...,4.959,5228.1,Withheld,0.50,0,1,0,1,0,0
54116,27,student,single,high.school,no,NaN,no,NaN,NaN,mon,...,1.354,5099.1,Withheld,1.00,0,0,0,1,0,0


In [81]:
column_mapping_str(data_all, col_name='default', prefix_name='default', data_name='data_all')
mapping_default = {'no': 0, 'yes': 1}
data_all['default_out'] = data_all['default'].map(mapping_default).fillna(0).astype(int)
data_all['default_out']

------------default每个项的分布--------------
no     26853
yes        3
Name: default, dtype: int64
--------------------------------------
mapping_default = {'no': 0, 'yes': 1}
data_all['default_out'] = data_all['default'].map(mapping_default).fillna(-99).astype(int)
data_all_default = pd.get_dummies(data_all['default_out'], prefix='default', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_default, index=data_all.index)], axis=1)
data_all = data_all.drop(labels='default', axis=1)
--------------------------------------


id
1        0
2        0
3        0
4        0
5        0
        ..
54114    0
54115    0
54116    0
54117    0
54118    0
Name: default_out, Length: 45307, dtype: int64

In [82]:
column_mapping_str(data_all, col_name='housing', prefix_name='housing', data_name='data_all')
mapping_housing = {'yes': 1, 'no': 0}
data_all['housing_out'] = data_all['housing'].map(mapping_housing).fillna(0).astype(int)
data_all['housing_out']

------------housing每个项的分布--------------
yes    17840
no     15326
Name: housing, dtype: int64
--------------------------------------
mapping_housing = {'yes': 0, 'no': 1}
data_all['housing_out'] = data_all['housing'].map(mapping_housing).fillna(-99).astype(int)
data_all_housing = pd.get_dummies(data_all['housing_out'], prefix='housing', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_housing, index=data_all.index)], axis=1)
data_all = data_all.drop(labels='housing', axis=1)
--------------------------------------


id
1        0
2        0
3        0
4        0
5        0
        ..
54114    1
54115    1
54116    0
54117    0
54118    1
Name: housing_out, Length: 45307, dtype: int64

In [83]:
column_mapping_str(data_all, col_name='loan', prefix_name='loan', data_name='data_all')
mapping_loan = {'no': 0, 'yes': 1}
data_all['loan_out'] = data_all['loan'].map(mapping_loan).fillna(0).astype(int)
data_all['loan_out']

------------loan每个项的分布--------------
no     27957
yes     5191
Name: loan, dtype: int64
--------------------------------------
mapping_loan = {'no': 0, 'yes': 1}
data_all['loan_out'] = data_all['loan'].map(mapping_loan).fillna(-99).astype(int)
data_all_loan = pd.get_dummies(data_all['loan_out'], prefix='loan', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_loan, index=data_all.index)], axis=1)
data_all = data_all.drop(labels='loan', axis=1)
--------------------------------------


id
1        0
2        0
3        0
4        0
5        1
        ..
54114    1
54115    0
54116    0
54117    0
54118    0
Name: loan_out, Length: 45307, dtype: int64

In [84]:
column_mapping_str(data_all, col_name='contact', prefix_name='contact', data_name='data_all')
mapping_contact = {'cellular': 2, 'telephone': 1}
data_all['contact_out'] = data_all['contact'].map(mapping_contact).fillna(0).astype(int)
data_all_contact = pd.get_dummies(data_all['contact_out'], prefix='contact', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_contact, index=data_all.index)], axis=1)
data_all


------------contact每个项的分布--------------
cellular     21604
telephone    12499
Name: contact, dtype: int64
--------------------------------------
mapping_contact = {'cellular': 0, 'telephone': 1}
data_all['contact_out'] = data_all['contact'].map(mapping_contact).fillna(-99).astype(int)
data_all_contact = pd.get_dummies(data_all['contact_out'], prefix='contact', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_contact, index=data_all.index)], axis=1)
data_all = data_all.drop(labels='contact', axis=1)
--------------------------------------


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,marital_out,education_1,education_2,education_3,default_out,housing_out,loan_out,contact_out,contact_1,contact_2
id,,,,,,,,,,,,,,,,,,,,,
1,NaN,housemaid,married,basic.4y,NaN,no,NaN,telephone,may,NaN,...,1,0,0,0,0,0,0,1,1,0
2,57,services,married,high.school,NaN,no,no,telephone,may,NaN,...,1,1,0,0,0,0,0,1,1,0
3,37,services,NaN,high.school,no,NaN,no,telephone,may,NaN,...,0,1,0,0,0,0,0,1,1,0
4,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,0,0,0,0,0,0,1,1,0
5,56,services,married,high.school,no,no,yes,NaN,may,mon,...,1,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54114,30,admin.,NaN,basic.6y,no,yes,yes,cellular,jul,thu,...,0,0,0,0,0,1,1,2,0,1
54115,39,admin.,NaN,NaN,no,yes,no,NaN,jul,NaN,...,0,1,0,0,0,1,0,0,0,0
54116,27,student,single,high.school,no,NaN,no,NaN,NaN,mon,...,0,1,0,0,0,0,0,0,0,0


In [85]:
fill_month = data_all.loc[~data_all['month'].isnull(), ['month','cons.price.idx']].drop_duplicates().sort_values(by=['month'])
data_all = data_all.merge(fill_month, on='cons.price.idx', how="left")
data_all.index.name='id'

In [86]:
#column_mapping_str(data_all, col_name='month', prefix_name='month', data_name='data_all')
mapping_month = {'may': 0, 'jul': 1, 'aug': 2, 'jun': 3, 'nov': 4, 'apr': 5,
                 'oct': 6, 'sep': 7, 'mar': 8, 'dec': 9}
data_all['month_out'] = data_all['month_y'].map(mapping_month).fillna(0).astype(int)
data_all_month = pd.get_dummies(data_all['month_out'], prefix='month', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_month, index=data_all.index)], axis=1)
data_all

,age,job,marital,education,default,housing,loan,contact,month_x,day_of_week,...,month_out,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,housemaid,married,basic.4y,NaN,no,NaN,telephone,may,NaN,...,0,0,0,0,0,0,0,0,0,0
1,57,services,married,high.school,NaN,no,no,telephone,may,NaN,...,0,0,0,0,0,0,0,0,0,0
2,37,services,NaN,high.school,no,NaN,no,telephone,may,NaN,...,0,0,0,0,0,0,0,0,0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,0,0,0,0,0,0,0,0,0,0
4,56,services,married,high.school,no,no,yes,NaN,may,mon,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45302,30,admin.,NaN,basic.6y,no,yes,yes,cellular,jul,thu,...,1,1,0,0,0,0,0,0,0,0
45303,39,admin.,NaN,NaN,no,yes,no,NaN,jul,NaN,...,1,1,0,0,0,0,0,0,0,0
45304,27,student,single,high.school,no,NaN,no,NaN,NaN,mon,...,0,0,0,0,0,0,0,0,0,0


In [87]:
column_mapping_str(data_all, col_name='day_of_week', prefix_name='day_of_week', data_name='data_all')
mapping_day_of_week = {'thu': 5, 'mon': 1, 'tue': 2, 'wed': 3, 'fri': 4}
data_all['day_of_week_out'] = data_all['day_of_week'].map(mapping_day_of_week).fillna(0).astype(int)
data_all_day_of_week = pd.get_dummies(data_all['day_of_week_out'], prefix='day_of_week', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_day_of_week, index=data_all.index)], axis=1)
data_all

------------day_of_week每个项的分布--------------
thu    7110
mon    7023
tue    6715
wed    6615
fri    6470
Name: day_of_week, dtype: int64
--------------------------------------
mapping_day_of_week = {'thu': 0, 'mon': 1, 'tue': 2, 'wed': 3, 'fri': 4}
data_all['day_of_week_out'] = data_all['day_of_week'].map(mapping_day_of_week).fillna(-99).astype(int)
data_all_day_of_week = pd.get_dummies(data_all['day_of_week_out'], prefix='day_of_week', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_day_of_week, index=data_all.index)], axis=1)
data_all = data_all.drop(labels='day_of_week', axis=1)
--------------------------------------


,age,job,marital,education,default,housing,loan,contact,month_x,day_of_week,...,month_6,month_7,month_8,month_9,day_of_week_out,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,housemaid,married,basic.4y,NaN,no,NaN,telephone,may,NaN,...,0,0,0,0,0,0,0,0,0,0
1,57,services,married,high.school,NaN,no,no,telephone,may,NaN,...,0,0,0,0,0,0,0,0,0,0
2,37,services,NaN,high.school,no,NaN,no,telephone,may,NaN,...,0,0,0,0,0,0,0,0,0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,0,0,0,0,1,1,0,0,0,0
4,56,services,married,high.school,no,no,yes,NaN,may,mon,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45302,30,admin.,NaN,basic.6y,no,yes,yes,cellular,jul,thu,...,0,0,0,0,5,0,0,0,0,1
45303,39,admin.,NaN,NaN,no,yes,no,NaN,jul,NaN,...,0,0,0,0,0,0,0,0,0,0
45304,27,student,single,high.school,no,NaN,no,NaN,NaN,mon,...,0,0,0,0,1,1,0,0,0,0


In [88]:
column_mapping_str(data_all, col_name='poutcome', prefix_name='poutcome', data_name='data_all')
mapping_poutcome = {'failure': 0, 'success': 1}
data_all['poutcome_out'] = data_all['poutcome'].map(mapping_poutcome).fillna(0).astype(int)

------------poutcome每个项的分布--------------
failure    3529
success    1146
Name: poutcome, dtype: int64
--------------------------------------
mapping_poutcome = {'failure': 0, 'success': 1}
data_all['poutcome_out'] = data_all['poutcome'].map(mapping_poutcome).fillna(-99).astype(int)
data_all_poutcome = pd.get_dummies(data_all['poutcome_out'], prefix='poutcome', drop_first=True)
data_all = pd.concat([data_all, pd.DataFrame(data_all_poutcome, index=data_all.index)], axis=1)
data_all = data_all.drop(labels='poutcome', axis=1)
--------------------------------------


In [ ]:
data_all['duration'] = data_all['duration'].astype(int)

In [89]:
mapping_y = {'no': 0, 'yes': 1}
data_all['y'] = data_all['y'].map(mapping_y)

In [90]:
data_all.columns

data_all.loc[data_all['y'].isin([0, 1]), ['age','info_rate', 'job_1',
       'job_3', 'marital_out', 'education_1', 'education_2', 'education_3',
       'default_out', 'housing_out', 'loan_out', 'contact_1',
       'contact_2', 'month_1', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'day_of_week_1', 'day_of_week_2', 'day_of_week_3',
       'day_of_week_4', 'day_of_week_5','duration', 'campaign', 'pdays',
       'previous', 'poutcome_out', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']].to_csv("./data/2_preprcossing_train.csv")

data_all.loc[data_all['y'].isnull(), ['age','info_rate', 'job_1',
       'job_3', 'marital_out', 'education_1', 'education_2', 'education_3',
       'default_out', 'housing_out', 'loan_out', 'contact_1',
       'contact_2', 'month_1', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'day_of_week_1', 'day_of_week_2', 'day_of_week_3',
       'day_of_week_4', 'day_of_week_5','duration', 'campaign', 'pdays',
       'previous', 'poutcome_out', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']].to_csv("./data/2_preprcossing_test.csv")

In [91]:
data_all

,age,job,marital,education,default,housing,loan,contact,month_x,day_of_week,...,month_7,month_8,month_9,day_of_week_out,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,poutcome_out
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,housemaid,married,basic.4y,NaN,no,NaN,telephone,may,NaN,...,0,0,0,0,0,0,0,0,0,0
1,57,services,married,high.school,NaN,no,no,telephone,may,NaN,...,0,0,0,0,0,0,0,0,0,0
2,37,services,NaN,high.school,no,NaN,no,telephone,may,NaN,...,0,0,0,0,0,0,0,0,0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,0,0,0,1,1,0,0,0,0,0
4,56,services,married,high.school,no,no,yes,NaN,may,mon,...,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45302,30,admin.,NaN,basic.6y,no,yes,yes,cellular,jul,thu,...,0,0,0,5,0,0,0,0,1,0
45303,39,admin.,NaN,NaN,no,yes,no,NaN,jul,NaN,...,0,0,0,0,0,0,0,0,0,0
45304,27,student,single,high.school,no,NaN,no,NaN,NaN,mon,...,0,0,0,1,1,0,0,0,0,0
